### **MlFlow Introduction**

MlFlow offers multiple services:
- **MlFlow Tracking** for logging models and training statistics, register and load models. Autologging can be enabled. 
- **MlFlow Tracking Server**: is a centralized HTTP server that allows you to access your experiments artifacts regardless of where you run your code.
- **MlFlow Registry** for registering a model in the MLflow model registry and how to retrieve registered models

#### **Setting**


MlFlow uses a HTTP server which needs to be defined by a host and a port. The **host** specifies the network interface (IP address) where the MLflow server will listen for incoming connection. By default, MLflow uses 127.0.0.1, which means the server is only accessible from the same machine (localhost). If you want the server to be accessible to other machines on your network, you can set it to 0.0.0.0, which binds it to all available network interfaces. The **port** defines the port number on which the MLflow Tracking Server runs.

In other words, we are working with another server, indeed it's possibile to query logged information by a post request.  

To specify mlflow server configuration:

mlflow server --host 0.0.0.0 --port 5000

You can start a tutorial and log models, experiments without a tracking server set up. With this mode, your experiment data and artifacts are saved directly under your current directory.

In this case, all IP within the network will be able to connect to the server. On the other side, for connection to the MlFlow UI through http://IP_DEL_SERVER:8080. 

mlflow server configuration can be defined: 
- Locally
- By Databricks Managed Services with limited quota. You can explore the mlflow services by the Databricks Workspace or import the notebook within Databricks. 
- By Cloud managed services

In [1]:
import mlflow

# If you're using a managed MLflow Tracking Server that is not provided 
# by Databricks, or if you're running a local tracking server, ensure 
# that you set the tracking server's uri using
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

#### **Logging and register through MlFlow**

You can also register a model manually through the UI. Moreover, tags can be assigned to the model like Staging or Production.

In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}
    
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result    

space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [4]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/03/17 09:07:17 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6:08 8s/step - loss: 37.6591 - root_mean_squared_error: 6.1367
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18.7866 - root_mean_squared_error: 4.2414 
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.6375 - root_mean_squared_error: 3.8341
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.6638 - root_mean_squared_error: 3.1069
46/46 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 10.5343 - root_mean_squared_error: 3.0863 - val_loss: 1.5236 - val_root_mean_squared_error: 1.2344

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 1.0813 - root_mean_squared_error: 1.0398
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3314 - root_mean_squared_error: 1.1534 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.3102 - root_mean_squared_error: 1.1443 - val_loss: 1.1238 - val_root_mean_squared_error: 1.0601

Epoch 3/3                                            

 1/46

In [5]:
# for inference (after registration by hand or with mlflow logging)

# loaded_model = mlflow.pyfunc.load_model(best_run.model_uri)
# pred = loaded_model.predict(test_x)
# pred

#### **Register and Load a Model**

We can register a model:
- Manually using the MlFlow UI
- By using API mlflow.log_model 

Moreover, aliases and tags can be attached to the registered model. 

Loading a registered model:
- mlflow.sklearn.load_model(f"runs:/{mlflow_run_id}/{run_relative_path_to_model}")
- mlflow.sklearn.load_model(f"models:/{model_name}/{model_version}")
- mlflow.sklearn.load_model(f"models:/{model_name}@{model_version_alias}")

#### **Serving models**

After registering a model, it can be served in order to be queried:

mlflow models serve -m "models:/wine-quality-best/1" --port 5002 --no-conda

It's better to run the tracking and serving in different machines for resources limitations within a production environment. 
Then, the model can be queried through a REST API request. 

In [14]:
import requests

# Define the MLflow serving endpoint
url = "http://localhost:5002/invocations"

# Define input data (modify based on your model's expected format)
payload = {"instances": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}  # Example input

# Set headers for JSON content
headers = {"Content-Type": "application/json"}

# Send request
response = requests.post(url, json=payload, headers=headers)

# Print response
print(response.json())  # Parsed JSON response from model


{'predictions': [[5.43813943862915]]}


#### **Serve with docker container**

Si potrebbe anche creare un container con all'interno il modello con tutta la sua configurazione:

mlflow models build-docker --model-uri "models:/wine-quality/1" --name "qs_mlops"

E runnarlo per l'utilizzo:

docker run -p 5002:8080 qs_mlops

Per cui la porta 5002 del localhost viene mappata sulla 8080 del container e quindi le richieste vengono reindirizzate al container. 

Interrogando il modello con delle chiamate REST API come prima si può ottenere la previsione. 

MlFlow server can be used in different ways:
    
- Locally: more secure for data
- Databricks Free Trial: leverages on Databricks platform for MlFlow functionalities
- Hosted Tracking service: managed solutions   